# 모델 평가

### 기존 개발 vs AI 개발
- 기존 개발 : 유닛 테스트, E2E 테스트 -> QA (품질보증, Quality Assurance)
- AI 개발 : 모델 평가 -> QA

### 모델 학습 및 배포 과정
- 데이터 취득 -> 모델 학습 -> 모델 평가 -> 모델 배포
- 모델 평가
  - 모델 검증
  - 모델 성능 지표 분석
  - 사용자 동작 테스트
  - 모델 모니터링 (off line (online 되기 전), online의 경우 실제 배포 후 성능지표)

### 모델 평가 과정
- Model Validation (학습)
- Model Evaluation
  - 모델 출력 분포 다양한 매트릭으로 정밀 분석
- Behavior Testing
  - 시나리오 테스트
  - 사용자 결과 테스트


### 주요 평가 메트릭
- 정확도 (Accuracy)
- F1 Score
  - 정밀도, 재현율 적절히 혼용
  - 모델의 정확도가 높더라도 샘플수가 적어서 편향될 수 있다.
- 정밀도 (Precision)
- 재현율 (Recall)
  - 모델 평가한 것 중 제대로 맞는 비율은?
  - 잘못 예측한 Positive를 줄이는 것이 목적
- 분류 성능 평가 지표
  - 실제 Positive 데이터 중 모델이 맞춘 비율은 어느정도?
  - 모델이 Positive 놓치는 것이 없도록 집중
- 드리프트 (Drift)
  - 모델이 잘 학습되었다 하더라도 실제 분포와 다를 수 있는데, 이 차이를 드리프트(Drift)라고 부른다.

## 분류 모델 학습 및 평가 실습

### 분류 모델
- 뉴스 카테고리 분류
- 과정
  - AG News 입력 -> BertForSequenceClassification -> 0 World, 1 Sports, 2 Business, 3 Sci/Tech
- BertForSequenceClassification
  - Bert는 경량데이터, 12만개중에서 1200개만 학습 돌릴 예정
- huggingface
  - 모델 및 데이터 등록 사이트 (GitHub와 비슷)

### 시나리오

```mermaid
A_1["데이터 셋 전처리"]-->B["모델 학습"]-->C["모델 평가"]
A_2["BERT 모델 다운로드"]-->B
C-->C_1["Accuracy 평가"]
C-->C_2["Precision, Recall, F1 Score 평가"]
C-->C_3["Confusion Matrix 평가"]
C-->C_4["ROC Curve 평가"]
```
- ROC Curve
  - Threshold에 따라 True Positive Rate(TPR)와 False Positive Rate(FPR)을 바탕으로 나타낸 모델 성능 그래프
  - 그래프 밑면의 넓이가 넓을 수록 모델 성능이 좋음
  - 대각선은 랜덤

In [ ]:
%pip install datasets
%conda install transformers
%pip install -U scikit-learn
# %pip install -U scikit-image
%pip install -U matplotlib
%conda install seaborn

In [3]:
import torch
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from torch.utils.data import DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from datasets import load_dataset
from tqdm import tqdm
from typing import TypedDict

from sklearn.metrics import auc, accuracy_score, confusion_matrix, precision_recall_fscore_support, roc_curve
from sklearn.preprocessing import label_binarize

/usr/local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
class DatasetItem(TypedDict):
    text:str
    label:str

def preprocess_data(dataset_item: DatasetItem) -> dict[str, torch.Tensor]:
    return tokenizer(dataset_item["text"], truncation=True, padding="max_length", return_tensors="pt")

train_dataset = dataset["train"].slect(range(1200)).map(preprocess_data, batched=True)
test_dataset = dataset["test"].slect(range(800)).map(preprocess_data, batched=True)

train_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])
test_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)

NameError: name 'TypedDict' is not defined